In [2]:
!pip3 install langchain_community

     ---------------------------------------- 2.2/2.2 MB 8.1 MB/s eta 0:00:00
     -------------------------------------- 124.8/124.8 kB 7.2 MB/s eta 0:00:00
     ------------------------------------- 973.6/973.6 kB 10.2 MB/s eta 0:00:00
     ------------------------------------- 310.4/310.4 kB 18.8 MB/s eta 0:00:00
     ---------------------------------------- 49.2/49.2 kB 2.6 MB/s eta 0:00:00
     -------------------------------------- 409.6/409.6 kB 8.5 MB/s eta 0:00:00
  Using cached packaging-23.2-py3-none-any.whl (53 kB)
     -------------------------------------- 138.6/138.6 kB 8.6 MB/s eta 0:00:00
  Using cached jsonpointer-2.4-py2.py3-none-any.whl (7.8 kB)
     ---------------------------------------- 1.9/1.9 MB 9.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
  Attempting uninstall: tenacity
    Found existing 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.1 requires flatbuffers<2,>=1.12, but you have flatbuffers 23.5.26 which is incompatible.
tensorflow 2.9.1 requires gast<=0.4.0,>=0.2.1, but you have gast 0.5.3 which is incompatible.
tensorflow 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.24.2 which is incompatible.
tensorflow 2.9.1 requires tensorflow-estimator<2.10.0,>=2.9.0rc0, but you have tensorflow-estimator 2.13.0 which is incompatible.


In [3]:
!python -m venv myenv

In [6]:
!myenv\Scripts\activate

In [7]:
!pip install langchain_community

In [8]:
from langchain_community.llms import Ollama

In [9]:
llm = Ollama(model="llama2")
llm.invoke("The first person on moon was ...")

'\nThe first person to walk on the moon was Neil Armstrong, an American astronaut who was part of the Apollo 11 mission in 1969. On July 20, 1969, Armstrong and fellow astronaut Edwin "Buzz" Aldrin landed their lunar module, Eagle, on the moon\'s surface and became the first humans to set foot on the moon. Armstrong famously declared, "That\'s one small step for man, one giant leap for mankind," as he stepped out of the lunar module and onto the moon\'s surface.'

### Read transaction data

In [10]:
import pandas as pd

In [11]:
df = pd.read_csv("transactions_2022_2023.csv")
df

,Date,Name / Description,Expense/Income,Amount (EUR)
0,2023-12-30,Belastingdienst,Expense,9.96
1,2023-12-30,Tesco Breda,Expense,17.53
2,2023-12-30,Monthly Appartment Rent,Expense,451.00
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46
4,2023-12-29,Selling Paintings,Income,13.63
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08
7,2023-12-22,Consulting,Income,541.57
8,2023-12-22,Aidsfonds,Expense,10.70
9,2023-12-20,Consulting,Income,2641.93


In [12]:
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

23

In [13]:
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 23]

In [14]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    print(response)

    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [15]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam',
                        llm)

['', 'Sure! Here are the appropriate categories for each expense:', '', '1. Spotify AB by Adyen - Entertainment', '2. ISS Catering Services De Meern - Food & Beverage', '3. Taxi Utrecht - Transportation', '4. Etos AMSTERDAM NLD - Grocery', '5. Bistro Bar Amsterdam - Food & Beverage']


,Transaction vs category,Transaction,Category
0,,,None
1,Sure! Here are the appropriate categories for ...,Sure! Here are the appropriate categories for ...,None
2,,,None
3,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
4,2. ISS Catering Services De Meern - Food & Bev...,2. ISS Catering Services De Meern,Food & Beverage
5,3. Taxi Utrecht - Transportation,3. Taxi Utrecht,Transportation
6,4. Etos AMSTERDAM NLD - Grocery,4. Etos AMSTERDAM NLD,Grocery
7,5. Bistro Bar Amsterdam - Food & Beverage,5. Bistro Bar Amsterdam,Food & Beverage


In [16]:
categories_df_all = pd.DataFrame()

for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

['Sure! Here are the appropriate categories for each expense:', '', '1. Belastingdienst - Taxes', '2. Tesco Breda - Groceries', '3. Monthly Appartment Rent - Housing', '4. Vishandel Sier Amsterdam - Food Delivery', '5. Selling Paintings - Art & Design', '6. Spotify Ab By Adyen - Entertainment', '7. Tk Maxx Amsterdam Da - Shopping', '8. Consulting - Professional Services', '9. Aidsfonds - Donations', '10. TLS Bv Inz Ov-Chipkaart - Transportation', '11. Etos Amsterdam - Convenience Store', '12. Beta Boulders Ams Amsterdam - Sports & Fitness', '13. Salary - Personal Expenses', '14. Bouldermuur Bv Amsterdam - Home Improvement', '15. Birtat Restaurant Amsterdam - Dining Out', '16. Freelancing - Professional Services', '17. Tikkie - Financial Services', '18. Blogging - Writing & Content Creation', '19. Taxi Utrecht - Transportation', '20. Apple Services - Technology', '21. Amazon Lux - Online Shopping', '22. Classpass* Monthly - Fitness & Wellness', '23. Audible Uk AdblCo/Pymt Gbr - Entertai

In [17]:
categories_df_all

,Transaction vs category,Transaction,Category
0,Sure! Here are the appropriate categories for ...,Sure! Here are the appropriate categories for ...,None
1,,,None
2,1. Belastingdienst - Taxes,1. Belastingdienst,Taxes
3,2. Tesco Breda - Groceries,2. Tesco Breda,Groceries
4,3. Monthly Appartment Rent - Housing,3. Monthly Appartment Rent,Housing
5,4. Vishandel Sier Amsterdam - Food Delivery,4. Vishandel Sier Amsterdam,Food Delivery
6,5. Selling Paintings - Art & Design,5. Selling Paintings,Art & Design
7,6. Spotify Ab By Adyen - Entertainment,6. Spotify Ab By Adyen,Entertainment
8,7. Tk Maxx Amsterdam Da - Shopping,7. Tk Maxx Amsterdam Da,Shopping
9,8. Consulting - Professional Services,8. Consulting,Professional Services


In [18]:
categories_df_all.to_csv("categories_df_all.csv", index=False)

In [19]:
unique_categories = categories_df_all["Category"].unique()
unique_categories

array([None, 'Taxes', 'Groceries', 'Housing', 'Food Delivery',
       'Art & Design', 'Entertainment', 'Shopping',
       'Professional Services', 'Donations', 'Transportation',
       'Convenience Store', 'Sports & Fitness', 'Personal Expenses',
       'Home Improvement', 'Dining Out', 'Financial Services',
       'Writing & Content Creation', 'Technology', 'Online Shopping',
       'Fitness & Wellness'], dtype=object)

In [20]:
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_20364\968977610.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_20364\968977610.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_20364\968977610.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documenta

In [21]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_20364\2440992137.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_20364\2440992137.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')


,Transaction vs category,Transaction,Category
2,1. Belastingdienst - Taxes,Belastingdienst,Taxes
3,2. Tesco Breda - Groceries,Tesco Breda,Groceries
4,3. Monthly Appartment Rent - Housing,Monthly Appartment Rent,Housing
5,4. Vishandel Sier Amsterdam - Food Delivery,Vishandel Sier Amsterdam,Food and Drinks
6,5. Selling Paintings - Art & Design,Selling Paintings,Art & Design
7,6. Spotify Ab By Adyen - Entertainment,Spotify Ab By Adyen,Entertainment
8,7. Tk Maxx Amsterdam Da - Shopping,Tk Maxx Amsterdam Da,Shopping
9,8. Consulting - Professional Services,Consulting,Services
10,9. Aidsfonds - Donations,Aidsfonds,Donations
11,10. TLS Bv Inz Ov-Chipkaart - Transportation,TLS Bv Inz Ov-Chipkaart,Transportation


In [22]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("transactions_2022_2023.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

,Date,Name / Description,Expense/Income,Amount (EUR),Transaction vs category,Transaction,Category
0,2023-12-30,Belastingdienst,Expense,9.96,1. Belastingdienst - Taxes,Belastingdienst,Taxes
1,2023-12-30,Tesco Breda,Expense,17.53,2. Tesco Breda - Groceries,Tesco Breda,Groceries
2,2023-12-30,Monthly Appartment Rent,Expense,451.00,3. Monthly Appartment Rent - Housing,Monthly Appartment Rent,Housing
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,4. Vishandel Sier Amsterdam - Food Delivery,Vishandel Sier Amsterdam,Food and Drinks
4,2023-12-29,Selling Paintings,Income,13.63,5. Selling Paintings - Art & Design,Selling Paintings,Art & Design
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19,6. Spotify Ab By Adyen - Entertainment,Spotify Ab By Adyen,Entertainment
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08,7. Tk Maxx Amsterdam Da - Shopping,Tk Maxx Amsterdam Da,Shopping
7,2023-12-22,Consulting,Income,541.57,8. Consulting - Professional Services,Consulting,Services
8,2023-12-22,Aidsfonds,Expense,10.70,9. Aidsfonds - Donations,Aidsfonds,Donations
9,2023-12-20,Consulting,Income,2641.93,8. Consulting - Professional Services,Consulting,Services


In [23]:
df.to_csv("transactions_2022_2023_categorized.csv", index=False)